# Tutorium Programmieren (Prof. Dr. Ralf Gerlich)
# Aufgabenblatt 9: Personaldatenbank
**Hinweis**: Die Lösungen der Aufgaben in diesem Aufgabenblatt werden im folgenden Aufgabenblatt erweitert.

Definieren Sie eine Klasse `Person` und eine Klasse `Personendatenbank` mit geeigneten Konstruktoren. Die Felder der Objekte dieser Klassen sollen mit der Notation `objekt.name` usw. lesbar, aber vor Schreibzugriff von außen geschützt sein.

Eine Person soll einen Nach- und Vornamen sowie ein Geburtsdatum haben. Implementieren Sie eine Methode `__str__`, die einen String mit einer gefälligen Darstellung mit Name, Vorname und Geburtsdatum zurückliefert. Für das Datum nutzen Sie die [Klasse `date` aus der `datetime`-Bibliothek](https://docs.python.org/3/library/datetime.html#datetime.date).

Eine Personendatenbank soll eine Liste von Personen enthalten. Die Liste soll von außen les-, aber nicht schreibbar sein.

Über die genannten Felder hinaus können Sie den Objekten zusätzliche Felder hinzufügen, die jedoch von außen nicht les- oder schreibbar sein dürfen.

Implementieren Sie für die Personendatenbank die folgenden Methoden:
- `leeren`: Leert die Liste der Personen.
- `einfügen(person)`: Fügt das Personenobjekt `person` hinten an die Liste der Personen an. 
- `lade(datei)`: Fügt die Einträge aus dem Dateiobjekt `datei` der Datenbank hinzu.
- `speichere(datei)`: Speichert alle Einträge in der Personendatenbank in das Dateiobjekt `datei`, und zwar so, dass sie mit `lade` wieder geladen werden können.
- `findePerson(name, vorname)`: Liefert das Personenobjekt aus der Datenbank, dessen Name und Vorname den Strings `name` und `vorname` entsprechen. Wenn keine solche Person existiert, soll `None` zurückgegeben werden.

Schreiben Sie in Jupyter ein Hauptprogramm, das die Verwaltung und Nutzung dieser Datenbank mit der folgenden Funktionalität erlaubt:
- Datenbank laden
- Datenbank speichern
- Person hinzufügen
- Person suchen

Implementieren Sie eine Hauptschleife, das ein entsprechendes Menü präsentiert, nach einer Auswahl fragt und dann die entsprechende Funktionalität ausführt, mit allen erforderlichen Nachfragen beim Nutzer oder der Nutzerin.

Testen Sie das Programm.

In [34]:
from datetime import date
import json

class Person:
    def __init__(self, fname, lname, bday):
        self.__fname = fname #schreibgeschützt durch __...
        self.__lname = lname
        self.__bday = date.fromisoformat(bday)

    @property # so nun als getter markiert
    def fname(self):
        return self.__fname

    @property
    def lname(self):
        return self.__lname

    @property
    def bday(self):
        return self.__bday

    def __str__(self):
        return f"{self.__fname} {self.__lname} (geb. {self.__bday})"
    
    def naechster_geburtstag(self):
        if self.bday.month >= date.today().month and self.bday.day >= date.today().day:
            nextbday = self.__bday.replace(year=date.today().year)
        else: nextbday = self.__bday.replace(year=date.today().year+1)
        return nextbday

class Personendatenbank:
    def __init__(self):
        self.__pliste = []
    
    def leeren(self):
        self.__pliste.clear()
    
    def einfuegen(self, nperson):
        self.__pliste.append(nperson)

    def lade(self, datei):
        jsonListe = json.load(datei)
        for line in jsonListe:
            neueperson = Person(
                line['fname'], 
                line['lname'], 
                line['bday']
            )
            self.einfuegen(neueperson)
        print(f"{len(jsonListe)} Person/en eingelesen.")

    def speichere(self, datei):
        zuSpeichern = []
        for person in self.__pliste:
            eintrag = {
                'fname': person.fname,
                'lname': person.lname,
                'bday': person.bday.isoformat()
            }
            zuSpeichern.append(eintrag)
        print(f"Speichere {len(zuSpeichern)} Person/en...")
        json.dump(zuSpeichern, datei, indent=4)
    
    def findePerson(self, sfname, slname):
        for p in self.__pliste:
            if p.fname == sfname and p.lname == slname:
                return p
        return None
    
    def naechste_geburtstage(self):
        naechste_geburtstage = self.__pliste
        naechste_geburtstage = sorted(naechste_geburtstage, key=Person.naechster_geburtstag)
        return naechste_geburtstage

def mainloop():
    datenbank = Personendatenbank()
    while True:
        print("\nWähle eine Option:")
        print("1. Datenbank laden")
        print("2. Datenbank speichern")
        print("3. Person hinzufügen")
        print("4. Person suchen")
        print("5. Datenbank leeren")
        print("6. Kommende Geburtstage anzeigen")
        print("7. Beenden")
        print("\n")

        auswahl = int(input("Auswahl: (1-7) "))
        if auswahl == 1:
            dateiname = input("Bitte kompletten Dateinamen zum Laden eingeben:")
            datei = open(dateiname, "r", encoding="utf-8")
            datenbank.lade(datei)
            datei.close()
            print(f"Datei in Datenbank geladen.")

        elif auswahl == 2:
            dateiname = input("Bitte kompletten Dateinamen zum Speichern eingeben:")
            datei = open(dateiname, "w", encoding="utf-8")
            datenbank.speichere(datei)
            datei.close()
            print("Datenbank in Datei gespeichert.")

        elif auswahl == 3:
            eingabe = input("Bitte Vorname, Nachname und Geburtsdatum im Format 'Vorname, Nachname, JJJJ-MM-TT' eingeben:").split(", ")
            neuePerson = Person(eingabe[0], eingabe[1], eingabe[2])
            datenbank.einfuegen(neuePerson)
            print(f"{neuePerson} eingefügt.")
        
        elif auswahl == 4:
            eingabe = input("Bitte Vorname und Nachname im Format 'Vorname, Nachname' eingeben:").split(", ")
            person = datenbank.findePerson(eingabe[0], eingabe[1])
            if person: print(f"Gefundene Person: {person}")
            else: print(f"Keine Person mit dem Namen {eingabe[0]} {eingabe[1]} gefunden.")
        
        elif auswahl == 5:
            datenbank.leeren()
            print("Datenbank geleert.")
        
        elif auswahl == 6:
            i = 0
            print("Die Personen, die als nächstes Geburtstag haben sind:")
            for p in datenbank.naechste_geburtstage():
                nAlter = p.naechster_geburtstag().year - p.bday.year
                print(f"- {p.fname} {p.lname}, hat am {p.naechster_geburtstag()} seinen {nAlter}. Geburtstag")
                i+=1
                if i == 5: break

        elif auswahl == 7: break
        else: print("Bitte valide Eingabe machen (1-6).")

mainloop()



Wähle eine Option:
1. Datenbank laden
2. Datenbank speichern
3. Person hinzufügen
4. Person suchen
5. Datenbank leeren
6. Kommende Geburtstage anzeigen
7. Beenden




4 Person/en eingelesen.
Datei in Datenbank geladen.

Wähle eine Option:
1. Datenbank laden
2. Datenbank speichern
3. Person hinzufügen
4. Person suchen
5. Datenbank leeren
6. Kommende Geburtstage anzeigen
7. Beenden


Die Personen, die als nächstes Geburtstag haben sind:
- eric zeller, hat am 2024-12-22 seinen 20. Geburtstag
- eric zeller, hat am 2025-07-14 seinen 21. Geburtstag
- eric zeller, hat am 2025-07-21 seinen 21. Geburtstag
- eric zeller, hat am 2025-12-20 seinen 21. Geburtstag

Wähle eine Option:
1. Datenbank laden
2. Datenbank speichern
3. Person hinzufügen
4. Person suchen
5. Datenbank leeren
6. Kommende Geburtstage anzeigen
7. Beenden




In [22]:
from datetime import date
date.today().__str__()

'2024-12-21'